In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from sklearn.cluster import MiniBatchKMeans
from sklearn.cluster import KMeans

from pythainlp.tokenize import word_tokenize
from string import punctuation

import numpy as np

import time

import collections

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

from scipy import stats

import math

import operator

def reset(df):
    cols = df.columns
    return df.reset_index()[cols]

print('OK !')

OK !


In [2]:
def canonicalize(string):
    normalized_tokens = list()
    a = word_tokenize(string, engine = 'newmm')
    for j in a:
        token = j.strip()
        #Add clean statement here 
        if len(token) > 1 and token not in set(punctuation) and token not in ['..','...','ๆๆ']:
            try:
                normalized_tokens.append(token.lower())
            except:
                normalized_tokens.append(token)
                pass
    return normalized_tokens

print('OK !')

OK !


In [3]:
# We pass in our tokenizer to the vectorizer object.
tf_vectorizer = CountVectorizer(tokenizer=canonicalize)
tfidf_vectorizer = TfidfVectorizer(tokenizer=canonicalize, sublinear_tf=True)
print('OK')

OK


In [4]:
data = pd.read_csv('data/clean_sample_text.csv')
data

published                                            content  \
0     11/07/20 00:09          โดน​ค่า ATS 14\r\n  บาท​กับ​หุ้น​อยู่​เลย   
1     11/07/20 00:51  ถ้า​ตาม​นิยาม​นี้\r\n  การ​ผูก​บัตร​เครดิต​กับ...   
2     11/07/20 01:30  ไร้​เงินสด เครดิต​ก็​ไม่​มี\r\n  หนี้​นอก​ระบบ...   
3     11/07/20 01:34        ถั่ว​ต้ม​เลย ไร้​เงินสด\r\n  ไร้​ของ​จะ​กิน   
4     11/07/20 03:30  อยากให้ลดข้อแม้ในการเข้าถึงแหล่งเงินทุนของสถาบ...   
...              ...                                                ...   
2919  18/09/20 20:24                                Suthatip Tongkasame   
2920  18/09/20 20:26   Pattarapong Sukkasem Yoke คิด​ถุง​เรา​เห​รอ​5555   
2921  18/09/20 21:00  ✨เก็บตกบางช่วงบางตอนของงาน Bangkok FinTech Fai...   
2922  18/09/20 21:41  อยาก​สะท้อน​ปัญหา​ทางการ​เงิน​และ​เศรษฐกิจ​จาก...   
2923  18/09/20 23:03  Nicharat\r\n  NanthachaiyalukPannaphat Nanthac...   

      engagement  len  
0            0.0   38  
1            0.0  184  
2            0.0   60  
3            0.0   40  
4            0.0  278  
...          ...  ...  
2919         0.0   19  
2920         0.0   48  
2921       454.0  479  
2922         0.0  106  
2923         0.0   51  

[2924 rows x 4 columns]

In [5]:
start = time.time()

tfidf_term_document_matrix = tfidf_vectorizer.fit_transform(data['content'])

print("Took " + "%.2f" % float(time.time() - start) + " sec")

Took 1.08 sec


In [6]:
start = time.time()

tf_term_document_matrix = tf_vectorizer.fit_transform(data['content'])

print("Took " + "%.2f" % float(time.time() - start) + " sec")

Took 1.01 sec


In [7]:
# Train cluster

start = time.time()

document_matrix = tfidf_term_document_matrix

num_clusters = 10

kmeans = KMeans(n_clusters=num_clusters, n_init=2, verbose=10)
kmeans.fit(document_matrix)

print("Took " + "%.2f" % float(time.time() - start) + " sec")

Initialization complete
Iteration 0, inertia 4901.306901954584
Iteration 1, inertia 2588.4868025163637
Iteration 2, inertia 2537.203210094027
Iteration 3, inertia 2529.056529134422
Iteration 4, inertia 2527.357761710986
Iteration 5, inertia 2526.6235236461125
Iteration 6, inertia 2526.20429487812
Iteration 7, inertia 2525.8924810275585
Iteration 8, inertia 2525.684503059527
Iteration 9, inertia 2524.8715700752236
Iteration 10, inertia 2523.8952880831057
Iteration 11, inertia 2523.5556459306126
Iteration 12, inertia 2523.310721352302
Iteration 13, inertia 2523.291118856248
Iteration 14, inertia 2523.2824421962073
Iteration 15, inertia 2523.2786321380727
Iteration 16, inertia 2523.2622406921255
Iteration 17, inertia 2523.213765460311
Iteration 18, inertia 2523.1209564996893
Iteration 19, inertia 2523.095925440719
Iteration 20, inertia 2523.0727975387395
Iteration 21, inertia 2523.0550771574444
Iteration 22, inertia 2523.0437627888664
Converged at iteration 22: strict convergence.
Initial

In [8]:
#General
import collections

text = data['content']

clus_dict = {}
stat_list = []
clus_dict_member = {}

# Group the posts by their cluster labels.
clustering = collections.defaultdict(list)
for idx, label in enumerate(kmeans.labels_):
    clustering[label].append(idx)


for cluster, indices in clustering.items():
    print("\nCluster:", cluster, " Num posts: ", len(indices),' Percent:', round(len(indices)*100/len(text), 2))
    cur_docs = 0
    
    clus_dict[cluster] = len(indices)*100/len(text)
    clus_dict_member[cluster] = len(indices)
    
    stat_list.append(len(indices)*100/len(text))
    
    for index in indices:
        if (cur_docs > 20):
            break
        post_contents = text.iloc[index].replace('\n', '')
        print(index, (post_contents[:75] + '..') if len(post_contents) > 75 else post_contents)
        cur_docs+=1


Cluster: 0  Num posts:  174  Percent: 5.95
  บาท​กับ​หุ้น​อยู่​เลย
  ไร้​ของ​จะ​กิน
15 แค่​สังสัยม.33นี่​ไม่​ได้​ผลก​ระ​ทบ​เลย​หรือ​เงิน​ไป​ไหน​หมด​เขา​กะ​เดือดร้..
  ดิฉั้น​ไม่​เคย​พก​เงิน​ติ​ว​ตัว​เลย​ค่ะ
  ประชาชน​ไม่มี​กำลัง​ซื้อ
  ลำพัง​เอา​ตัว​เอง​ให้​รอด​ยัง​ลำบาก​เลย..
48 แต่งงาน​แล้ว​ไม่​มี​ปัญญา​เลี้ยง​ลูก​ที่​เกิด​มา​ก็​อย่า​แต่ง​เลย​เป็น​ภาระ..
  ปัญหา​ความ​เป็น​อยู่​จะ​ตาม​มา ถ้า​แต่งงาน..
  จะ​ปล่อย​ผลิตภัณย์ตั..
  แบ​ง​ค์หัว​สี​บานเย็น ขอ​จะ​ๆ 3% เลย ค่า​ดำเนิน​การ
225 ส่วนตัว​มอง​ว่า​ยัง​ไม่​ถึง​ก้น​เหว​เลย​ฮะ
  เคย​มา​ประชุม​ร่วม​กัน​กับ​ภาค​ธุรกิจ​บ้างม..
 ..
264 กำลัง​มา​เลย​ครับ​เรื่อง​นี้
272 ยัง​เอา​มา​ลง​อับอาย​ไร้​น้ำยา
273 5​แสน​ล้าน​พึ่ง​ปล่อย​ไป​แสน​ล้าน​เอา​อะไร​มา​คิด​มา​อวด​ผล​งาน
275 เอา​ไป​ปล่อย​คลอง​ไหน​ไม่​เห็น​มี​ใคร​ได้​กู้​เลย
  แต่​พนักงาน​ใน​ระบบ​ประกัน​ภาค​รัฐ ยัง​ไม่​ได้​เงิน เยียว ยา น🙄
294 CBDC​ สกุล​เงิน​ดิจิตอล​จะเข้าเว็บเทรดสกุลเงิน​ประเทศ​ไหม่
299 steak token ได้​จะ​น่า​สนใจ​มาก​เลย
  บ่อย..ถึง​บอก​ให้​ทำ​เงิน​ดิจิตอ​ลมาใช้ก่อ​น..55+

Clu

In [9]:
stat_list

[5.950752393980848,
 14.124487004103967,
 10.465116279069768,
 5.198358413132695,
 37.65389876880985,
 6.395348837209302,
 9.678522571819425,
 3.9671682626538987,
 0.6839945280437757,
 5.882352941176471]

In [10]:
stats.describe(stat_list)

DescribeResult(nobs=10, minmax=(0.6839945280437757, 37.65389876880985), mean=10.0, variance=108.29476702079684, skewness=2.067755941652068, kurtosis=3.265891610859059)

In [11]:
import operator

sorted_x = sorted(clus_dict.items(), key=operator.itemgetter(1))

for each in sorted_x[::-1]:
    print(each[0], round(each[1],2))

2 37.65
6 14.12
5 10.47
3 9.68
1 6.4
0 5.95
7 5.88
8 5.2
4 3.97
9 0.68


In [12]:
km = kmeans
X = tfidf_term_document_matrix


text = data['content']
nearest_points = 15

for j in range(num_clusters):
    d = km.transform(X)[:, j]
    ind = np.argsort(d)[:nearest_points]
    
    print('Cluster:', j, ' Percent:',clus_dict[j])
    for i in ind:
        print(i, text.iloc[i])
        print('___________________________________________________________________________________________')
    print()
    print()
    print('___________________________________________________________________________________________')
    print('___________________________________________________________________________________________')    
    print('___________________________________________________________________________________________')  
    print()
    print()

Cluster: 0  Percent: 5.950752393980848
1936 มา​บ้าน​เลย
___________________________________________________________________________________________
2647 ไม่​เห็น​ได้​เลย
___________________________________________________________________________________________
387 เศรษฐกิจ​จะ​ไป​ต่อ​ได้​แค่​ไหน ?
  ขึ้น​อยู่​กับ​ธนาคาร​แห่ง​ประเทศ​ไทย​จะ​พิมพ์​เงิน​ออก​มา​หล่อเลี้ยง​ประเทศ​ได้​แค่​ไหน
  เพราะ​รัฐบาล​ไม่​มี​เงิน​ใช้ และ​ไม่​สามารถ​พิมพ์​เงิน​ออก​มา​ใช้
  ดั่ง​ธนาคาร​แห่ง​ประเทศ​ไทย
___________________________________________________________________________________________
2207 เดือน​กว่า​ยัง​ไม่​มี​เจ้าหน้าที่​ติดต่อ​กลับ​มา​เลย​ครับ
___________________________________________________________________________________________
1981 เรา​ยัง​ไม่​ประกาศ​เลย
___________________________________________________________________________________________
2311 63 ยัง​หยุด​ไม่​ครบ​เลย 64
  มา​แล้ว
___________________________________________________________________________________________
264 กำลัง​มา